# Safari Challenge

In this challenge, you must use what you've learned to train a convolutional neural network model that classifies images of animals you might find on a safari adventure.

## Explore the data

The training images you must use are in the **/safari/training** folder. Run the cell below to see an example of each image class, and note the shape of the images (which indicates the dimensions of the image and its color channels).

In [1]:
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import skimage.io
from pathlib import Path

# The images are in the data/shapes folder
data_path = Path("data/safari/training")

# Get the class names
classes = [d.name for d in data_path.iterdir()]
classes.sort()
print(len(classes), "classes:")
print(classes)

# Show the first image in each folder
fig = make_subplots(
    1, len(classes), subplot_titles=[str(i) for i in range(len(classes))]
)
titles = {}
i = 0
for sub_dir in data_path.iterdir():
    i += 1
    img_file = next(sub_dir.iterdir())
    img = skimage.io.imread(img_file)
    img_shape = img.shape
    img_fig = px.imshow(img)
    titles[str(i - 1)] = f"{img_file.name} : {str(img_shape)}"
    fig = fig.add_traces(img_fig.data, 1, i)
fig.for_each_annotation(lambda a: a.update(text=titles[a.text]))
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
display(fig)


4 classes:
['elephant', 'giraffe', 'lion', 'zebra']


Now that you've seen the images, use your preferred framework (PyTorch or TensorFlow) to train a CNN classifier for them. Your goal is to train a classifier with a validation accuracy of 95% or higher.

Add cells as needed to create your solution.

> **Note**: There is no single "correct" solution. Sample solutions are provided in [05 - Safari CNN Solution (PyTorch).ipynb](05%20-%20Safari%20CNN%20Solution%20(PyTorch).ipynb) and [05 - Safari CNN Solution (TensorFlow).ipynb](05%20-%20Safari%20CNN%20Solution%20(TensorFlow).ipynb).

### Préparation du modèle de base, pour le transfer learning

In [2]:
from keras.applications.resnet import ResNet50

# Your Code to train a CNN model...
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
display(base_model.summary())


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

None

### Préparation des données

In [3]:
from keras.preprocessing.image import ImageDataGenerator

data_folder = 'data/safari/training'
pretrained_size = (224,224)
batch_size = 30

print("Getting Data...")
datagen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.25) # hold back 30% of the images for validation

print("Preparing training dataset...")
train_generator = datagen.flow_from_directory(
    data_folder,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator = datagen.flow_from_directory(
    data_folder,
    target_size=pretrained_size, # resize to match model expected input
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

Getting Data...
Preparing training dataset...
Found 300 images belonging to 4 classes.
Preparing validation dataset...
Found 96 images belonging to 4 classes.
class names:  ['elephant', 'giraffe', 'lion', 'zebra']


### Création de la couche de prédiction

In [4]:
from keras import applications
from keras import Model
from keras.layers import Flatten, Dense

# Freeze the already-trained layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Create prediction layer for classification of our images
x = base_model.output
x = Flatten()(x)
prediction_layer = Dense(len(classnames), activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=prediction_layer)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Now print the full model, which will include the layers of the base model plus the dense layer we added
display(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

None

### Entraînement du modèle

In [5]:
# Train the model over 5 epochs
num_epochs = 5
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

Epoch 1/5
10/10 [==============================] - 80s 8s/step - loss: 4.2060 - accuracy: 0.4367 - val_loss: 1.3877 - val_accuracy: 0.6000
Epoch 2/5
10/10 [==============================] - 60s 6s/step - loss: 1.0027 - accuracy: 0.7100 - val_loss: 0.1842 - val_accuracy: 0.9444
Epoch 3/5
10/10 [==============================] - 60s 6s/step - loss: 0.3683 - accuracy: 0.8333 - val_loss: 0.2290 - val_accuracy: 0.9333
Epoch 4/5
10/10 [==============================] - 64s 6s/step - loss: 0.1335 - accuracy: 0.9600 - val_loss: 0.0533 - val_accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 57s 6s/step - loss: 0.0572 - accuracy: 0.9867 - val_loss: 0.0516 - val_accuracy: 1.0000


### Observation des métriques

In [6]:
epoch_nums = list(range(1,num_epochs+1))
fig = make_subplots(1, 2)
fig.add_trace(go.Scatter(x=epoch_nums, y=history.history["loss"], name='training'), row=1, col=1)
fig.add_trace(go.Scatter(x=epoch_nums, y=history.history["val_loss"], name='validation'), row=1, col=1)
fig.update_xaxes(title='epoch', col=1)
fig.update_yaxes(title='loss', col=1)

fig.add_trace(go.Scatter(x=epoch_nums, y=history.history["accuracy"], name='training'), row=1, col=2)
fig.add_trace(go.Scatter(x=epoch_nums, y=history.history["val_accuracy"], name='validation'), row=1, col=2)
fig.update_xaxes(title='epoch', col=2)
fig.update_yaxes(title='accuracy', col=2)
display(fig)

### Évaluation du modèle

In [7]:
# Tensorflow doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn
import numpy as np
from sklearn.metrics import confusion_matrix

print("Generating predictions from validation data...")
# Get the image and label arrays for the first batch of validation data
x_test = validation_generator[0][0]
y_test = validation_generator[0][1]

# Use the model to predict the class
class_probabilities = model.predict(x_test)

# The model returns a probability value for each class
# The one with the highest probability is the predicted class
predictions = np.argmax(class_probabilities, axis=1)

# The actual labels are hot encoded (e.g. [0 1 0], so get the one with the value 1
true_labels = np.argmax(y_test, axis=1)

# Plot the confusion matrix
cm = confusion_matrix(true_labels, predictions)
px.imshow(
    cm,
    text_auto=True,
    x=classnames,
    y=classnames,
    color_continuous_scale="blues",
    labels={"x": "Predicted Animal", "y": "Actual Animal"},
)


Generating predictions from validation data...
1/1 [==============================] - 5s 5s/step


## Save your model

Add code below to save your model's trained weights.

In [8]:
# Code to save your model
modelFileName = 'models/safari_classifier.tf'
model.save(modelFileName)
del model  # deletes the existing model variable
print('model saved as', modelFileName)

INFO:tensorflow:Assets written to: models/safari_classifier.tf\assets


INFO:tensorflow:Assets written to: models/safari_classifier.tf\assets


model saved as models/safari_classifier.tf


## Use the trained model

Now that we've trained your model, modify the following code as necessary to use it to predict the classes of the provided test images.

In [9]:
from keras import models
from skimage.transform import resize

# Function to predict the class of an image
def predict_image(classifier: Model, image: np.ndarray):
    import numpy

    # Default value
    index = 0

    # !!Add your code here to predict an image class from your model!!
    # The model expects a batch of images as input, so we'll create an array of 1 image
    imgfeatures = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

    # Use the model to predict the image class
    class_probabilities = classifier.predict(imgfeatures)
    
    # Find the class predictions with the highest predicted probability
    index = int(np.argmax(class_probabilities, axis=1)[0])

    # Return the predicted index
    return index


# Load your model
model = models.load_model(modelFileName)

# The images are in the data/shapes folder
test_data_path = Path("data/safari/test")

# Show the test images with predictions
fig = make_subplots(
    1, len(classes), subplot_titles=[str(i) for i in range(len(classes))]
)
titles = {}
i = 0
for img_file in test_data_path.iterdir():
    i += 1
    img = skimage.io.imread(img_file)
    img = resize(img, pretrained_size)
    # Get the image class prediction
    index = predict_image(model, img)
    img_fig = px.imshow(img)
    titles[str(i - 1)] = classes[index]
    fig = fig.add_traces(img_fig.data, 1, i)
fig.for_each_annotation(lambda a: a.update(text=titles[a.text]))
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
display(fig)

1/1 [==============================] - 0s 261ms/step


Hopefully, your model predicted all four of the image classes correctly!